In [21]:
import pandas as pd
import ast

# Movies

In [22]:
url_movies = "https://drive.google.com/uc?export=download&id=1Rp7SNuoRnmdoQMa5LWXuK4i7W1ILblYb"


In [23]:
df= pd.read_csv(url_movies)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12608\2317543060.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(url_movies)


In [ ]:
# Eliminamos columnas que no necesitamos
df.drop(["video","adult","homepage","imdb_id","poster_path","original_title"],axis=1,inplace=True)

In [ ]:
df.dropna(subset=["release_date"],inplace=True)

In [ ]:
df.reset_index(inplace=True)


In [ ]:
df.drop(["index"],axis=1,inplace=True)

In [ ]:
# Normalizamos columna
df["belongs_to_collection"]=df["belongs_to_collection"].apply(lambda x:ast.literal_eval(x) if pd.notna(x) else {})

In [ ]:
normalize= pd.json_normalize(df["belongs_to_collection"])

In [ ]:
df = pd.concat([df, normalize["name"]],axis=1)

In [ ]:
df.drop(["belongs_to_collection"],axis=1,inplace=True)

In [ ]:
df.rename(columns= {"name" : "belongs_to_collection"}, inplace= True)

### Aplicamos la funcion ast convertir las cadenas a listas de diccionarios

In [ ]:
df['spoken_languages'] = df['spoken_languages'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

In [ ]:
df['production_countries'] = df['production_countries'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

In [ ]:
df['production_companies'] = df['production_companies'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

In [ ]:
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

### TRANSFORMAR TIPOS DE DATOS CORRESPONDIENTES 

In [ ]:
df.info()

In [ ]:
df["budget"] = pd.to_numeric(df["budget"], errors= "coerce")

In [ ]:
df.dropna(subset=["budget"],inplace= True)

In [ ]:
df["id"]=df["id"].astype("int64")

In [ ]:
df["popularity"]=df["popularity"].astype(float)

In [ ]:
df.dropna(subset=["budget"],inplace= True)

In [ ]:
df["release_date"]= pd.to_datetime(df["release_date"], errors= "coerce")

In [ ]:
# eliminar valores duplicados 
df.drop_duplicates(subset="id", inplace= True)

In [ ]:
df.reset_index(inplace= True)

In [ ]:
df.drop(columns="index",inplace= True)

In [ ]:
df.shape

# Creditos

In [19]:
df1 = pd.read_parquet("Datasets/Movies/creditos.parquet")

##### Crear lista de actores

In [ ]:
# Aplicar una función para convertir las cadenas a listas de diccionarios
df1["cast"] = df1['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [ ]:
def extract_name_gender(cast_list):
    return [cast_member['name'] for cast_member in cast_list]

In [ ]:
df1["cast"]=df1["cast"].apply(extract_name_gender)

#### crear lista de director

In [ ]:
df1["crew"]=df1["crew"].apply(lambda x : ast.literal_eval(x) if isinstance(x,str) else x)

In [ ]:
def director(crew1):
    lista=[]
    for member in crew1:
        if member["job"]=="Director":
            lista.append(member["name"])
    return lista

In [ ]:
df1["crew"]=df1['crew'].apply(director)

In [ ]:
df1.rename({"crew" : "director"},axis=1, inplace= True)

In [ ]:
df1.head(5)

#### Unificamos los DataFrames y eliminamos duplicados

In [ ]:
df_final = pd.merge(df, df1, on='id', how='inner')

In [ ]:
df_final.drop_duplicates(subset="id",inplace=True)

### agregar mes y dias para las consultas

In [ ]:
meses = {
    1: 'enero', 
    2: 'febrero', 
    3: 'marzo', 
    4: 'abril', 
    5: 'mayo', 
    6: 'junio', 
    7: 'julio', 
    8: 'agosto', 
    9: 'septiembre', 
    10: 'octubre', 
    11: 'noviembre', 
    12: 'diciembre'
}

In [ ]:
dias = {
    0: "lunes",
    1: 'martes', 
    2: 'miercoles', 
    3: 'jueves', 
    4: 'viernes', 
    5: 'sabado', 
    6: 'domingo', 
}

In [ ]:
df_final["mes"]

In [ ]:
df_final["mes"] = df_final["release_date"].dt.month.map(meses)

In [ ]:
df_final["dia"] = df_final["release_date"].dt.day_of_week.map(dias)

### descargar dataset y verificar

In [ ]:
df_final.to_csv("movies_limpios.csv", index=False)

In [ ]:
df_final.to_parquet("movies.parquet", index=False)

In [ ]:
df_verificado= pd.read_parquet("movies.parquet")

In [ ]:
df_verificado